# RAG mit Azure AI Agent Service & Semantic Kernel

Dieses Codebeispiel zeigt, wie man einen Azure AI-Agenten für Retrieval-Augmented Generation (RAG) mit dem `Azure AI Agent Service` und `Semantic Kernel` erstellt und verwaltet. Der Agent verarbeitet Benutzeranfragen basierend auf dem abgerufenen Kontext und liefert entsprechend präzise Antworten.


SQLite-Version beheben  
Falls Sie auf den Fehler stoßen:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Kommentieren Sie diesen Codeblock am Anfang Ihres Notebooks aus:


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Importieren von Paketen
Der folgende Code importiert die erforderlichen Pakete:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Retrieval-Augmented Generation mit Semantic Kernel & Azure AI Agent Service

Dieses Beispiel zeigt, wie der **Azure AI Agent Service** verwendet werden kann, um **Retrieval-Augmented Generation (RAG)** durch die Kombination eines Sprachmodells mit domänenspezifischem Kontext aus einem hochgeladenen Dokument durchzuführen.

### Funktionsweise

1. **Dokument-Upload**: Eine Markdown-Datei (document.md), die Informationen enthält (Contosos Reiseversicherungspolice), wird in den Agentendienst hochgeladen.

2. **Erstellung eines Vektorspeichers**: Das Dokument wird in einem Vektorspeicher indexiert, um eine semantische Suche über dessen Inhalte zu ermöglichen.

3. **Agentenkonfiguration**: Ein Agent wird mit dem `gpt-4o` Modell und den folgenden strikten Anweisungen eingerichtet:
   - Beantworte Fragen nur basierend auf den abgerufenen Inhalten aus dem Dokument.
   - Lehne es ab, Fragen zu beantworten, die außerhalb des Themenbereichs liegen.

4. **Integration des File Search Tools**: Das `FileSearchTool` wird mit dem Agenten registriert, wodurch das Modell während der Inferenz relevante Ausschnitte aus dem indexierten Dokument suchen und abrufen kann.

5. **Interaktion mit Nutzern**: Nutzer können Fragen stellen. Wenn relevante Informationen im Dokument gefunden werden, generiert der Agent eine fundierte Antwort.  
   Falls nicht, antwortet der Agent ausdrücklich, dass das Dokument keine ausreichenden Informationen enthält.


### Hauptfunktion



Stellen Sie sicher, dass Sie zuerst `az login` mit der Azure CLI ausführen, damit der richtige Authentifizierungskontext bereitgestellt wird, während Sie `DefaultAzureCredential` verwenden. Der Azure AI Agent Service verwendet keine API-Schlüssel.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Sure! Please provide the markdown file you'd like me to translate.



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
